In [ ]:
!pip3 install ipywidgets xgboost scikit-learn pandas numpy --user

^C


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import pickle
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Leer datos
df = pd.read_csv("database.csv")

# Extraer habitaciones y baños del título
df['n_beds'] = df['title'].str.extract(r'(\d+)\s*beds').fillna(0).astype(int)
df['n_baths'] = df['title'].str.extract(r'(\d+)\s*baths?').fillna(0).astype(int)

# Codificar ubicación
le = LabelEncoder()
df['location_encoded'] = le.fit_transform(df['location'])

# Preparar features
features = ['location_encoded', 'mts', 'n_beds', 'n_baths']
X = df[features]
y = df['price']

print("Datos cargados exitosamente!")

In [ ]:
# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar Random Forest
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)
rf_model.fit(X_train_scaled, y_train)

# Evaluar modelo
train_score = rf_model.score(X_train_scaled, y_train)
test_score = rf_model.score(X_test_scaled, y_test)

print(f'Puntuación en entrenamiento: {train_score:.4f}')
print(f'Puntuación en prueba: {test_score:.4f}')

# Guardar modelo
with open('modelo_casas.pkl', 'wb') as file:
    pickle.dump({
        'modelo': rf_model,
        'scaler': scaler,
        'label_encoder': le
    }, file)

In [ ]:
def predecir_precio():
    try:
        # Solicitar datos al usuario
        print("\nIngresa los datos de la casa:")
        
        # Ubicación
        print("\nUbicaciones disponibles:")
        print("1: Norte")
        print("2: Sur")
        print("3: Centro")
        loc_num = int(input("Selecciona el número de la ubicación (1-3): "))
        location = ['Norte', 'Sur', 'Centro'][loc_num-1]
        
        # Otros datos
        metros = float(input("Metros cuadrados: "))
        habitaciones = int(input("Número de habitaciones: "))
        banos = int(input("Número de baños: "))
        
        # Cargar modelo
        with open('modelo_casas.pkl', 'rb') as file:
            data = pickle.load(file)
        
        modelo = data['modelo']
        scaler = data['scaler']
        le = data['label_encoder']
        
        # Preparar datos para predicción
        location_encoded = le.transform([location])[0]
        X_pred = np.array([[location_encoded, metros, habitaciones, banos]])
        X_pred_scaled = scaler.transform(X_pred)
        
        # Realizar predicción
        prediccion = modelo.predict(X_pred_scaled)[0]
        
        # Mostrar resultados
        print("\nResultados de la predicción:")
        print(f"Ubicación: {location}")
        print(f"Metros cuadrados: {metros}")
        print(f"Habitaciones: {habitaciones}")
        print(f"Baños: {banos}")
        print(f"\nPrecio estimado: ${prediccion:,.2f}")
        
        # Mostrar rango de confianza (±10%)
        print(f"Rango de precios esperado:")
        print(f"Mínimo: ${prediccion * 0.9:,.2f}")
        print(f"Máximo: ${prediccion * 1.1:,.2f}")
        
        return prediccion
    
    except Exception as e:
        print(f"Error en la predicción: {str(e)}")
        return None

In [ ]:
# Ejecutar predicción
predecir_precio()